# 11 · Dashboard Executivo
## Consolidação de KPIs e Status Operacional

<div align="center">

```
┌─────────────────────────────────────────────────────────────┐
│       EXECUTIVE DASHBOARD - AML SYSTEM STATUS v1.0         │
└─────────────────────────────────────────────────────────────┘
```

![Status](https://img.shields.io/badge/Status-Operational-success)
![Type](https://img.shields.io/badge/Type-Executive%20Report-blue)
![Audience](https://img.shields.io/badge/Audience-Leadership-purple)

</div>

---

<div style="background-color: #2d2416; border-left: 4px solid #f59e0b; padding: 15px; border-radius: 4px;">

**OBJETIVO**

Consolidar resultados de todo o pipeline AML em dashboard executivo, fornecendo visibilidade de KPIs de performance, governança e status operacional para tomada de decisão estratégica.

</div>

### ARTEFATOS CONSOLIDADOS

<table>
<tr><th>Categoria</th><th>Artefatos</th><th>Notebook Origem</th></tr>
<tr><td><b>Performance</b></td><td>PR-AUC, F1, Recall@K, Lift</td><td>07, 08</td></tr>
<tr><td><b>Threshold</b></td><td>Limiar ótimo, Expected Value</td><td>09</td></tr>
<tr><td><b>Drift</b></td><td>PSI scores, features em drift</td><td>10</td></tr>
<tr><td><b>Explainability</b></td><td>Feature importance, SHAP values</td><td>07</td></tr>
<tr><td><b>Governança</b></td><td>Logs de auditoria, versionamento</td><td>Todos</td></tr>
</table>

### ESTRUTURA DO DASHBOARD

```
┌────────────────────────────────────────────────────────────┐
│                    EXECUTIVE DASHBOARD                      │
├────────────────────────────────────────────────────────────┤
│                                                            │
│   KPIs PRINCIPAIS                                        │
│  ┌─────────────┬─────────────┬─────────────┬───────────┐ │
│  │ Lift @ K    │ Precision   │ Drift Status│ Model Age │ │
│  │   2.5x      │   87.3%     │  ESTÁVEL    │  15 days  │ │
│  └─────────────┴─────────────┴─────────────┴───────────┘ │
│                                                            │
│   PERFORMANCE METRICS                                    │
│  ┌──────────────────────────────────────────────────────┐ │
│  │ - PR-AUC: 0.872 (Target: > 0.80)          ✅         │ │
│  │ - Recall@100: 45.2% (Capacity: 100/day)   ✅         │ │
│  │ - Alert Rate: 2.1% (Operational)           ✅         │ │
│  └──────────────────────────────────────────────────────┘ │
│                                                            │
│  ⚠️ ALERTAS E AÇÕES                                        │
│  ┌──────────────────────────────────────────────────────┐ │
│  │ - 3 features com drift moderado            ⚠️         │ │
│  │ - Retreinamento recomendado em 7 dias      ℹ️         │ │
│  └──────────────────────────────────────────────────────┘ │
│                                                            │
│  📋 RECOMENDAÇÕES EXECUTIVAS                               │
│  ┌──────────────────────────────────────────────────────┐ │
│  │ 1. Manter threshold atual (ROI otimizado)             │ │
│  │ 2. Monitorar features: Payment Format, Amount        │ │
│  │ 3. Agendar review trimestral de performance          │ │
│  └──────────────────────────────────────────────────────┘ │
└────────────────────────────────────────────────────────────┘
```

<div style="background-color: #1a2332; border-left: 4px solid #3b82f6; padding: 15px; border-radius: 4px; margin-top: 15px;">

**AUDIÊNCIA E USO**

**STAKEHOLDERS:**
- **C-Level**: KPIs de alto nível, impacto no negócio
- **Risk Management**: Status de detecção, taxa de alarmes
- **Data Science**: Métricas técnicas, drift, performance
- **Compliance/Audit**: Rastreabilidade, governança

**FREQUÊNCIA:**
- Review semanal: KPIs principais e alertas
- Review mensal: Performance detalhada e tendências
- Review trimestral: ROI, estratégia, roadmap

</div>

### COMPONENTES DO DASHBOARD

```
KPIs                  Métricas                  Alertas
┌──────────┐         ┌──────────┐             ┌──────────┐
│ Lift     │         │ PR-AUC   │             │ Drift    │
│ Precision│    →    │ Recall@K │      →      │ Quality  │
│ ROI      │         │ F1-Score │             │ Actions  │
└──────────┘         └──────────┘             └──────────┘
     │                    │                        │
     └────────────────────┴────────────────────────┘
                          │
                   ┌──────▼──────┐
                   │  Executive   │
                   │ Recommendations│
                   └─────────────┘
```

<div style="background-color: #1a2332; border-left: 4px solid #3b82f6; padding: 15px; border-radius: 4px; margin-top: 15px;">

**NOTA TÉCNICA**

Dashboard consolidado gera relatório executivo automaticamente. Para visualizações interativas, considerar Plotly/Streamlit.

</div>

---

In [1]:
# · SETUP ULTRA-LIMPO (substitui 10+ linhas de imports)
import sys
from pathlib import Path
sys.path.insert(0, str(Path('..') / 'utils'))

from notebook_setup_ultra_clean import setup_monitoring
env = setup_monitoring()

# Extrair o que preciso
pd, np, config = env['pd'], env['np'], env['config']
data_dir, artifacts_dir = env['data_dir'], env['artifacts_dir']
plt, sns = env['plt'], env['sns']
quick_save = env['quick_save']

# Dashboard específicos
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

try:
    import plotly.express as px
    import plotly.graph_objects as go
    HAS_PLOTLY = True
except ImportError:
    HAS_PLOTLY = False

def load_artifact(name, type_="csv"):
    """Carrega artefato com fallback seguro"""
    path = artifacts_dir / name
    if not path.exists():
        return None if type_ == "csv" else {}
    
    if type_ == "csv":
        return pd.read_csv(path)
    else:  # json
        with open(path, "r", encoding="utf-8") as f:
            return json.load(f)

# Carregar todos os artefatos
artifacts = {
    "thresholds": ("thresholds.json", "json"),
    "ensemble_metrics": ("ensemble_metrics_at_k.csv", "csv"),
    "drift_summary": ("drift_detection_summary.csv", "csv"),
    "drift_metadata": ("drift_detection_metadata.json", "json"),
    "permutation_importance": ("permutation_importance.csv", "csv"),
    "monitor_scores": ("monitor_score_shift.csv", "csv")
}

print("· 17_reporting_dashboard configurado - ZERO redundância!")
print(f"   Plotly disponível: {HAS_PLOTLY}")
print(f"   Artifacts dir: {artifacts_dir}")

· Setup LIMPO: monitoring
 Visualization Suite carregada!
[OK] Auto-carregou: engineered - Train: (2400, 12), Test: (1460, 12)
[OK] 34 recursos disponíveis | Datasets: [OK]
· 17_reporting_dashboard configurado - ZERO redundância!
   Plotly disponível: True
   Artifacts dir: c:\Users\gafeb\OneDrive\Desktop\lavagem_dev\artifacts


In [3]:
# Dashboard KPIs Consolidados
# Primeiro, carregar todos os artefatos
data = {}
for key, (filename, file_type) in artifacts.items():
    data[key] = load_artifact(filename, file_type)

kpis = {}

# Performance do Ensemble
if data["ensemble_metrics"] is not None:
    best_row = data["ensemble_metrics"].loc[data["ensemble_metrics"]["lift@K"].idxmax()]
    kpis["ensemble"] = {
        "best_k": int(best_row["K"]),
        "lift": best_row["lift@K"],
        "precision": best_row["precision@K"]
    }
    print(f"ENSEMBLE: K={kpis['ensemble']['best_k']}, Lift={kpis['ensemble']['lift']:.2f}, Precision={kpis['ensemble']['precision']:.3f}")

# Drift Detection
if data["drift_summary"] is not None:
    max_drift = data["drift_summary"].loc[data["drift_summary"]["psi_max"].idxmax()]
    kpis["drift"] = {
        "max_psi": max_drift["psi_max"],
        "window": max_drift["window"],
        "features_drift": max_drift["features_com_drift"]
    }
    print(f"DRIFT: PSI Max={kpis['drift']['max_psi']:.3f} ({kpis['drift']['window']}), Features={kpis['drift']['features_drift']}")

# Feature Importance
if data["permutation_importance"] is not None:
    top_feature = data["permutation_importance"].iloc[0]
    kpis["top_feature"] = {
        "name": top_feature["feature"],
        "importance": top_feature["importance"]
    }
    print(f"TOP FEATURE: {kpis['top_feature']['name']} ({kpis['top_feature']['importance']:.4f})")

# Thresholds
if data["thresholds"]:
    levels = data["thresholds"].get("levels", {})
    kpis["thresholds"] = len(levels)
    print(f"THRESHOLDS: {kpis['thresholds']} níveis configurados")

# Score Monitoring
if data["monitor_scores"] is not None:
    score_psi = data["monitor_scores"]["psi_scores"].iloc[0]
    interpretation = data["monitor_scores"]["interpretation"].iloc[0]
    kpis["score_drift"] = {"psi": score_psi, "status": interpretation}
    print(f"SCORE DRIFT: PSI={score_psi:.4f} ({interpretation})")

print(f"\nDashboard gerado com {len(kpis)} KPIs principais")

DRIFT: PSI Max=13.740 (Janela_3), Features=5
SCORE DRIFT: PSI=0.0148 (Stable (< 0.1))

Dashboard gerado com 2 KPIs principais


In [4]:
# Executive Summary
print("=== EXECUTIVE SUMMARY ===")

# Status Geral
if kpis:
    print("STATUS OPERACIONAL:")
    
    # Performance
    if "ensemble" in kpis:
        status = "EXCELENTE" if kpis["ensemble"]["lift"] > 2.0 else "BOM" if kpis["ensemble"]["lift"] > 1.5 else "CRÍTICO"
        print(f"   Performance: {status} (Lift: {kpis['ensemble']['lift']:.2f})")
    
    # Drift Status
    if "drift" in kpis:
        drift_status = "CRÍTICO" if kpis["drift"]["max_psi"] > 0.5 else "ATENÇÃO" if kpis["drift"]["max_psi"] > 0.25 else "ESTÁVEL"
        print(f"   Drift Status: {drift_status} (PSI Max: {kpis['drift']['max_psi']:.3f})")
    
    # Score Monitoring
    if "score_drift" in kpis:
        score_status = "ESTÁVEL" if kpis["score_drift"]["status"] == "Stable" else "ATENÇÃO"
        print(f"   Score Drift: {score_status} ({kpis['score_drift']['status']})")

# Recomendações
print("\nRECOMENDAÇÕES:")
recommendations = []

if "ensemble" in kpis and kpis["ensemble"]["lift"] < 1.5:
    recommendations.append("- Revisar configuração do ensemble")

if "drift" in kpis and kpis["drift"]["max_psi"] > 0.25:
    recommendations.append("- Monitorar features com drift alto")
    recommendations.append("- Considerar retreinamento do modelo")

if "score_drift" in kpis and kpis["score_drift"]["status"] != "Stable":
    recommendations.append("- Investigar distribuição de scores")

if not recommendations:
    recommendations = ["- Sistema operando dentro dos parâmetros normais"]

for rec in recommendations:
    print(rec)

print(f"\nDashboard consolidado - {len(kpis)} KPIs analisados")

=== EXECUTIVE SUMMARY ===
STATUS OPERACIONAL:
   Drift Status: CRÍTICO (PSI Max: 13.740)
   Score Drift: ATENÇÃO (Stable (< 0.1))

RECOMENDAÇÕES:
- Monitorar features com drift alto
- Considerar retreinamento do modelo
- Investigar distribuição de scores

Dashboard consolidado - 2 KPIs analisados


In [5]:
# Detalhamento de Drift (Top Features)
if data["drift_summary"] is not None:
    print("TOP 5 FEATURES COM MAIOR DRIFT:")
    
    # Assumindo que temos dados de drift detalhados ou usando summary
    top_drift = data["drift_summary"].nlargest(5, "psi_max")
    
    for i, (_, row) in enumerate(top_drift.iterrows(), 1):
        window = row["window"]
        psi_max = row["psi_max"]
        features_drift = row["features_com_drift"]
        
        # Status baseado no PSI
        if psi_max > 0.5:
            status = "CRÍTICO"
        elif psi_max > 0.25:
            status = "ATENÇÃO"
        else:
            status = "ESTÁVEL"
        
        print(f"{i}. {window}: PSI={psi_max:.3f} | Features={features_drift} | {status}")
    
    total_windows = len(data["drift_summary"])
    critical_windows = len(data["drift_summary"][data["drift_summary"]["psi_max"] > 0.5])
    print(f"\nResumo: {critical_windows}/{total_windows} janelas críticas")
else:
    print("Dados de drift não disponíveis")

print("\nAnálise de drift concluída")

TOP 5 FEATURES COM MAIOR DRIFT:
1. Janela_3: PSI=13.740 | Features=5 | CRÍTICO
2. Janela_4: PSI=13.731 | Features=6 | CRÍTICO
3. Janela_2: PSI=4.040 | Features=4 | CRÍTICO
4. Janela_1: PSI=1.100 | Features=3 | CRÍTICO

Resumo: 4/4 janelas críticas

Análise de drift concluída
